<a href="https://colab.research.google.com/github/Tclack88/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/LS_DS_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


## data cleaning, and preparation

In [0]:
import pandas as pd

In [2]:
url = "https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv"
df = pd.read_csv(url)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.dtypes # "object" type for "TotalCharges" makes no sense, further investigation shows there are blank column entries

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
# df.isna().sum() # returns 0, so no nans

df = df.drop(columns=['customerID'])
blank_indices = df[df.TotalCharges == ' '].index # 11 cols have blanks here
df = df.drop(index=blank_indices)
df.TotalCharges = df.TotalCharges.astype('float64')
YN_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

YN_map = {'Yes': 1, 'No' : 0}

for col in YN_cols:
  df[col] = df[col].map(YN_map)

df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,Male,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,0
2,Male,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,Male,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1


In [5]:
!pip install category_encoders

     |████████████████████████████████| 102kB 13.5MB/s 


In [0]:
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [0]:
X = df.drop(columns='Churn')
y = df.Churn

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [0]:
pipeline = Pipeline([('encoder', ce.OneHotEncoder(use_cat_names=True)),
                      ('normalizer', MinMaxScaler())])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

In [13]:
X_train.shape

(5274, 41)

## Make Neural Network

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV 
import numpy as np

In [0]:
np.random.seed(88)

def create_model():
  """ required for KerasClassifier """
  model = Sequential()
  model.add(Dense(20, activation='relu', input_dim=41))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


model = KerasClassifier(build_fn=create_model, verbose=0)

### Find Top contributors to batch size

In [25]:
param_grid = {'batch_size': [10,20,50,80,100],
              'epochs': [20]}

search = GridSearchCV(model, param_grid=param_grid, verbose=1, cv=3)
search.fit(X_train,y_train)    

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.9min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f072a30e9e8>,
             iid='warn', n_jobs=None,
             param_grid={'batch_size': [10, 20, 50, 80, 100], 'epochs': [20]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [35]:
print(f'BEST - score: {search.best_score_}  params: {search.best_params_}')
a = search.cv_results_['split0_test_score']
b = search.cv_results_['split1_test_score']
c = search.cv_results_['split2_test_score']

(a + b + c ) / 3
# 80, 20 and 50 got the best scores respectively, they will be carried on to the next round

BEST - score: 0.7993932565053304  params: {'batch_size': 80, 'epochs': 20}


array([0.79199849, 0.79844519, 0.79806598, 0.79939326, 0.79730755])

### Find best activation function for output

In [39]:
def create_model(activation='sigmoid'):
  """ required for KerasClassifier """
  model = Sequential()
  model.add(Dense(20, activation='relu', input_dim=41))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation=activation))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'activation': ['relu', 'sigmoid','tanh', 'softmax', 'softplus'],
              'epochs': [20]}

search = GridSearchCV(model, param_grid=param_grid, verbose=1, cv=3)
search.fit(X_train,y_train)  

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f06c012f588>,
             iid='warn', n_jobs=None,
             param_grid={'activation': ['relu', 'sigmoid', 'tanh', 'softmax',
                                        'softplus'],
                         'epochs': [20]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [41]:
print(f'BEST - score: {search.best_score_}  params: {search.best_params_}')
a = search.cv_results_['split0_test_score']
b = search.cv_results_['split1_test_score']
c = search.cv_results_['split2_test_score']
(a + b + c ) / 3
# Best activation functions in order: Sigmoid, Softplus, relu

BEST - score: 0.8016685644785563  params: {'activation': 'sigmoid', 'epochs': 20}


array([0.7899128 , 0.80166856, 0.76602199, 0.27038301, 0.79086081])

### Find Best Dropout Fractions

In [43]:
def create_model(fraction=.2):
  """ required for KerasClassifier """
  model = Sequential()
  model.add(Dense(20, activation='relu', input_dim=41))
  model.add(Dropout(fraction))
  model.add(Dense(10, activation='relu'))
  model.add(Dropout(fraction))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'fraction': [0, .05, .1, .15, .2],
              'epochs': [20]}

search = GridSearchCV(model, param_grid=param_grid, verbose=1, cv=3)
search.fit(X_train,y_train)  

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f06bf177400>,
             iid='warn', n_jobs=None,
             param_grid={'epochs': [20], 'fraction': [0, 0.05, 0.1, 0.15, 0.2]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [44]:
print(f'BEST - score: {search.best_score_}  params: {search.best_params_}')
a = search.cv_results_['split0_test_score']
b = search.cv_results_['split1_test_score']
c = search.cv_results_['split2_test_score']
(a + b + c ) / 3
# best dropout fractions: .2, 0, .1

BEST - score: 0.7993932565053304  params: {'epochs': 20, 'fraction': 0.2}


array([0.79673872, 0.7967387 , 0.79673872, 0.79446342, 0.79939326])

### perform a CV with the top 3 from each category individually

In [45]:
def create_model(fraction=.2, activation='sigmoid'):
  """ required for KerasClassifier """
  model = Sequential()
  model.add(Dense(20, activation='relu', input_dim=41))
  model.add(Dropout(fraction))
  model.add(Dense(10, activation='relu'))
  model.add(Dropout(fraction))
  model.add(Dense(1, activation=activation))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [80, 20, 50],
              'activation': ['sigmoid', 'softmax', 'relu'],
              'fraction': [0, .1, .2],
              'epochs': [20]}

search = GridSearchCV(model, param_grid=param_grid, verbose=1, cv=3)
search.fit(X_train,y_train)  

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed: 20.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f06bde7ce10>,
             iid='warn', n_jobs=None,
             param_grid={'activation': ['sigmoid', 'softmax', 'relu'],
                         'batch_size': [80, 20, 50], 'epochs': [20],
                         'fraction': [0, 0.1, 0.2]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [46]:
print(f'BEST - score: {search.best_score_}  params: {search.best_params_}')
a = search.cv_results_['split0_test_score']
b = search.cv_results_['split1_test_score']
c = search.cv_results_['split2_test_score']
(a + b + c ) / 3

BEST - score: 0.8010997374852499  params: {'activation': 'sigmoid', 'batch_size': 80, 'epochs': 20, 'fraction': 0.1}


array([0.79863481, 0.80109974, 0.79882441, 0.79484264, 0.79427379,
       0.7965491 , 0.7965491 , 0.794653  , 0.79275693, 0.27038301,
       0.27038301, 0.27038301, 0.27038301, 0.27038301, 0.27038301,
       0.27038301, 0.27038301, 0.27038301, 0.77284793, 0.77512326,
       0.76602201, 0.79560107, 0.78877511, 0.77948427, 0.79010238,
       0.78081155, 0.76431553])

### keep best combination above and try different epochs

In [47]:
def create_model():
  """ required for KerasClassifier """
  model = Sequential()
  model.add(Dense(20, activation='relu', input_dim=41))
  model.add(Dropout(.1))
  model.add(Dense(10, activation='relu'))
  model.add(Dropout(.1))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [80],
              'epochs': [20,50,100,200,500]}

search = GridSearchCV(model, param_grid=param_grid, verbose=1)
search.fit(X_train,y_train)  

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 11.8min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f06bde7ecc0>,
             iid='warn', n_jobs=None,
             param_grid={'batch_size': [80], 'epochs': [20, 50, 100, 200, 500]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [48]:
print(f'BEST - score: {search.best_score_}  params: {search.best_params_}')
a = search.cv_results_['split0_test_score']
b = search.cv_results_['split1_test_score']
c = search.cv_results_['split2_test_score']
(a + b + c ) / 3

BEST - score: 0.7986348072687784  params: {'batch_size': 80, 'epochs': 50}


array([0.79598027, 0.79863481, 0.79446342, 0.7883959 , 0.78403487])

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?